In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install timm

In [ ]:
import pandas as pd
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import cv2
import torch
import timm
from albumentations import Compose, Normalize
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class TestDataset(Dataset):
    
    def __init__(self, df, image_path, transform=None):
        self.df = df
        self.image_path = image_path
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):

        img = cv2.imread(
            self.image_path + str(self.df.image_id[idx]) + '.png'
        )
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            img = self.transform(image=img)['image']
        img = torch.from_numpy(img.transpose((2, 0, 1))).float()
            
        return img

In [ ]:
import tqdm
def test_model(model, val_loader):    
    """
    Inference using the model on the test set
    
    Parameters:
        model (torch.nn.Module): The model to be trained.
        val_loader (torch.utils.data.DataLoader): Dataloader object for validation.
    Return: 
        preds (np.array): An array with shape (num_samples, 15).
    """
    model.eval()
    
    preds = []
    with torch.no_grad():
        tk = tqdm(val_loader, total=len(val_loader), position=0, leave=True)
        for idx, (imgs) in enumerate(tk):
            imgs_valid = imgs.to(device)
            output_valid = model(imgs_valid)
            
            valid_pred = torch.sigmoid(output_valid).detach().cpu().numpy()
         
            preds.append(valid_pred)

        preds = np.concatenate(preds,axis=0)
            
    return preds

In [ ]:
!pip install -U albumentations

     |████████████████████████████████| 102 kB 32.6 MB/s 
     |████████████████████████████████| 47.8 MB 85.7 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import os
from tqdm.auto import tqdm
import pandas as pd
from albumentations import (
    Compose,
    Normalize,
    ShiftScaleRotate,
    RandomBrightnessContrast,
    MotionBlur,
    CLAHE,
    HorizontalFlip
)
csv_path = '/content/drive/Shareddrives/efficienet/test_data/test.csv'
test = pd.read_csv(csv_path)
image_path='/content/drive/Shareddrives/efficienet/test_data/test/'
#num =len(os.listdir(image_path))
#print(num)
test_transform = Compose([Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0)])
                         
valset = TestDataset(
            test,
            image_path = image_path,
            transform=test_transform
        )
val_loader = torch.utils.data.DataLoader(
            valset, batch_size=4, shuffle=False, num_workers=1
        )


In [ ]:
import torch.nn as nn
model=timm.create_model('tf_efficientnet_b4_ns', pretrained=True)
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 1)

In [ ]:
from tqdm.auto import tqdm
PATH='/content/drive/Shareddrives/efficienet/output train 35 póc/Fold 0 model with val_acc 0.9916666666666668.pth'
preds = np.zeros((3000,1))
model.load_state_dict(torch.load(PATH))
model.to(device)
predict = test_model(model,val_loader)
preds[:,0]=predict[:,0]
#preds[:,1]=predict[:,1]
df_preds = pd.DataFrame(preds, columns=np.arange(0,1)).assign(image_id = test['image_id'])
df_preds.to_csv("/content/drive/Shareddrives/efficienet/2_classfil_efficienet.csv",index=False) 

  0%|          | 0/750 [00:00<?, ?it/s]

In [ ]:
p = np.array([[1,2,3],[4,5,6],[7,8,9]])
p[:,1]

array([2, 5, 8])

[0]
